In [1]:
# Calling libraries:
from __future__ import division
%matplotlib inline
import numpy as np, time, matplotlib.pyplot as plt, math, pandas, numpy.random as npr, multiprocessing as mp
from pylab import plot, show, legend
from time import time
from scipy.stats import *
from tqdm import trange
from ecology_functions import *

In [2]:
T = 25
I = 5   # number of locations
J = 3   # number of species
K = 2   # number of latent factors

In [3]:
lmbda = npr.randn(J,K)
alpha = npr.randn(J)
c = 0
phi = 0.5
logsigmasq = 0
x_0 = npr.randn(I,K)

theta = [alpha, lmbda, c, phi, logsigmasq]
Y, X = simulate_data(x_0, T, J, theta)

In [4]:
def f1(n_particles) :
    return block_pf(Y, x_0, n_particles, theta, calc_grad=True)
def f2(n_particles) :
    return bootstrap_PF_grad(x_0, n_particles, theta, Y, calc_grad=True)

n_particles = 1_000
rep = 1000

pool = mp.Pool(10)
results1 = pool.map(f1, [n_particles for n_particles in [n_particles]*rep])
results2 = pool.map(f2, [n_particles for n_particles in [n_particles]*rep])
pool.close()

logNC = np.zeros((rep,2))
alpha_grad = np.zeros((rep,2,J))
lmbda_grad = np.zeros((rep,2,J,K))
c_grad = np.zeros((rep,2))
phi_grad = np.zeros((rep,2))
logsigmasq_grad = np.zeros((rep,2))

for r in range(rep) :
    alpha_grad[r,0], lmbda_grad[r,0], c_grad[r,0], phi_grad[r,0], logsigmasq_grad[r,0] = results1[r][1][:]
    alpha_grad[r,1], lmbda_grad[r,1], c_grad[r,1], phi_grad[r,1], logsigmasq_grad[r,1] = results2[r][1][:]
    logNC[r,0], logNC[r,1] = results1[r][0], results2[r][0]

In [7]:
np.mean(logNC,0)

array([-248.00593279, -338.14186729])

In [8]:
np.var(logNC,0)

array([  2.8633466 , 139.57192297])

In [5]:
print(np.var(c_grad,0))
print(np.var(lmbda_grad,0))
print(np.var(phi_grad,0))
print(np.var(logsigmasq_grad,0))

[  3.0563794  154.95168727]
[[[  1.40736353   2.53464999]
  [  1.98843889   3.59685457]
  [  0.84617704   1.71034298]]

 [[ 83.20163349 102.95416656]
  [100.74971145 134.21086397]
  [ 44.92385451  55.66198679]]]
[ 40.00742894 730.07769037]
[  5.21283183 108.99689849]


In [6]:
print(np.mean(c_grad,0))
print(np.mean(lmbda_grad,0))
print(np.mean(phi_grad,0))
print(np.mean(logsigmasq_grad,0))

[-38.67488814  -1.20171077]
[[[ 49.24509278  52.64589722]
  [ 40.56625965 -15.43987376]
  [ 47.35411864  16.22096097]]

 [[ 17.50422512  14.6935659 ]
  [  9.38447678 -14.24399795]
  [ 17.98122158   5.48682663]]]
[-276.34439403 -207.73928256]
[131.48563098 111.0083866 ]
